In [32]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [14]:
# Load the data
data = pd.read_excel('VCT_DATASET.xlsx')
test_data = pd.read_excel('Test_VCT_DATASET.xlsx').dropna(how='any',axis=0)

In [16]:
# Select features and target variable
features = ['rounds', 'rating', 'kills_per_round', 'assists_per_round',
            'first_kills_per_round', 'first_deaths_per_round', 'headshot_percentage',
            'clutch_success_percentage', 'total_kills', 'total_deaths', 'total_assists',
            'total_first_kills', 'total_first_deaths']

target_variable = 'average_combat_score'

In [17]:
# Split variables
X_train = data[features]
X_test = test_data[features]
y_train = data[target_variable]
y_test = test_data[target_variable]

In [19]:
# Initialize the XGBoost Regressor
model = XGBRegressor()

In [20]:
# Train the model
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [29]:
# Make predictions on the test set
y_pred = model.predict(X_test.to_numpy())

In [37]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'mean Squared Error: {mse}')

mean Squared Error: 98.51821786988859
